# HANDWRITTEN MULTIPLE-CHOICE TEST EVALUATOR
An implementation of automated handwritten multiple-choice test evaluation, employing OpenCV for image preprocessing and PyTorch for training a convolutional neural network on the EMNIST dataset for image classification.

## IMPORT MODULES AND SET GLOBALS

In [ ]:
!pip install gradio

In [2]:
# For downloading model weights
import os

# For image preprocessing
import cv2
import numpy as np
from sklearn.cluster import KMeans

# For the image classifier
import torch
import torch.nn as nn
import torch.nn.functional as F

# For the interface
import gradio as gr

In [3]:
# Globals:
# Define constants
PREDICTION_TO_STRING = ["A", "B", "C", "D"]

# Define variables
# For preprocessing
image_width = 1125
image_height = 1500
thresh_block_size = 11
thresh_constant = 5
line_length = 5000

# For ROI extraction
min_contour_width = 15
min_contour_height = 15
max_contour_width = 100
max_contour_height = 100
y_epsilon = 25
number_of_columns = 2

## THE CLASSIFIER
Prepare the ABCD Classifier Model.

In [4]:
# Download the model's state dictionary from repository
GITHUB_URL = "https://raw.githubusercontent.com/CorvusBrachyrhynchos/Handwritten-Multiple-Choice-Test-Corrector/main/abcd_classifier.pth"
MODEL_PATH = "/content/model.pth"
os.system(f"wget {GITHUB_URL} -O {MODEL_PATH}")
model_state_dict = torch.load(MODEL_PATH)

In [5]:
# Define the model class
class ABCDClassifier(nn.Module):

  def __init__(self):
    super(ABCDClassifier, self).__init__()

    self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(32 * 4 * 4, 128)
    self.fc2 = nn.Linear(128, 4)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 32 * 4 * 4)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)

    return F.softmax(x, dim=1)

In [ ]:
# Load the model
model = ABCDClassifier()
model.load_state_dict(model_state_dict)

## THE FUNCTIONS
Define the functions for interfacing, image preprocessing, region of interest (ROI) extraction, and image classification.

### Interface Functions

In [7]:
# The main function used for the interface.
# Takes in an array of strings (image paths), and an array of chars (correction key)
# Returns three arrays: image scores, score analysis, item analysis
def correct_image_set(image_paths_array, correction_key):
  image_scores = [] # Length is len(image_paths_array)
  score_analysis = [0] * (len(correction_key) + 1)
  item_analysis = [0] * len(correction_key)

  print(f"CORRECTION KEY: {correction_key}")

  for i in range(len(image_paths_array)):
    correct_image(image_paths_array[i], image_scores, score_analysis, item_analysis, correction_key, i + 1)

  print("\nAFTER EVALUATION AND ANALYSIS")
  print(f"Image Scores: {image_scores}")
  print(f"Score Analysis: {score_analysis}")
  print(f"Item Analysis: {item_analysis}")

  return image_scores, score_analysis, item_analysis


# A helper function for readability
# Takes in an image path, the three arrays to be modified, an array of chars, and an integer
# Void return value, modifies the three arrays directly
def correct_image(img_path, img_scr_arr, scr_ana_arr, itm_ana_arr, correction_key, student_number):
  score = 0
  item_answers = [] # An array of strings

  # Begin processing for ROI extraction
  base_image = cv2.imread(img_path)
  resized_image = cv2.resize(base_image, (image_width, image_height))
  image_for_contour_finding, ocr_image, processing_error = preprocess(resized_image)
  if processing_error: # Check for exception
    invalid_num_of_items(img_scr_arr)
    return

  rois, extraction_error = extract_rois(image_for_contour_finding, ocr_image, len(correction_key))
  if extraction_error: # Check for exception
    invalid_num_of_items(img_scr_arr)
    return

  # Classify the ROIs
  for roi in rois:
    item_answers.append(classify_roi(roi))

  print(f"\nSTUDENT {student_number}:")
  print(f"Item Answers ({len(item_answers)}): {item_answers}\n")

  if len(item_answers) != len(correction_key): # Check for exception (extra layer of safety)
    invalid_num_of_items(img_scr_arr)
    return

  # Evaluate and analyze
  for i in range(len(correction_key)):
    if item_answers[i] == correction_key[i] or correction_key[i] == "X":
      score += 1
      itm_ana_arr[i] += 1
  img_scr_arr.append(score)
  scr_ana_arr[score] += 1


# Function for error logic: sets the score to -1 (-1 score means invalid image)
# Takes in the array to be modified (score array)
# Void return value, modifies the array directly
def invalid_num_of_items(score_array):
  score_array.append(-1)
  print("INVALID: NUM OF ITEM ANSWERS != NUM OF ITEMS IN CORRECTION KEY")

### Image Preprocessing Functions

In [8]:
# Takes in an image (the base image)
# Returns an image made for contour detection, an image for OCR, and a boolean for error handling
def preprocess(input_image):
  # ------------------------------------ FOR OCR IMAGE------------------------------------ #
  gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
  threshed_gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, thresh_block_size, thresh_constant)
  opened = cv2.bitwise_not(cv2.morphologyEx(cv2.bitwise_not(threshed_gray), cv2.MORPH_OPEN, np.ones((2,2),np.uint8), iterations=1))
  image_for_ocr = remove_lines(opened)

  # ----------------------------------- FOR HEADER MASK ----------------------------------- #
  blur = cv2.GaussianBlur(gray, (9,9), 0)
  threshed_blur = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, thresh_block_size, thresh_constant)
  removed_lines = remove_lines(threshed_blur, 6)
  intensely_dilated = cv2.dilate(cv2.bitwise_not(removed_lines), cv2.getStructuringElement(cv2.MORPH_RECT, (2000, 40)), iterations=1)

  # Determine divider y positions
  black_ys = np.where(np.any(intensely_dilated == 0, axis=1))[0].tolist()
  to_be_removed = []
  for i in range(1, len(black_ys)):
    if black_ys[i] - black_ys[i - 1] == 1:
      to_be_removed.append(i)
  stripe_positions = [black_ys[idx] for idx in range(len(black_ys)) if not idx in to_be_removed]

  # Determine which stripe is the header border
  num_of_stripe_positions = len(stripe_positions)
  header_border = 0
  match num_of_stripe_positions:
    case 1:
      header_border = stripe_positions[0]
    case 2:
      header_border = stripe_positions[0] if stripe_positions[1] > 500 else stripe_positions[1]
    case _:
      found = False
      for stripe_position in stripe_positions:
        if stripe_position > 100 and stripe_position < 500:
          header_border = stripe_position
          found = True
          continue
      if not found: # If there is no stripe, consider the image invalid
        return [0], [0], True

  # Create a mask based on header border
  mask = np.ones(intensely_dilated.shape, dtype=np.uint8) * 255
  mask[header_border:, :] = 0

  # ---------------------------------- FOR CONTOUR IMAGE ---------------------------------- #
  masked_removed_lines = cv2.bitwise_or(removed_lines, mask)
  dilated = cv2.dilate(cv2.bitwise_not(masked_removed_lines), cv2.getStructuringElement(cv2.MORPH_RECT, (6, 6)), iterations=1)
  image_for_countour = cv2.Canny(dilated, 100, 200)
  return image_for_countour, image_for_ocr, False



In [9]:
# A helper function for the preprocess function
# Takes in an image, and optionally line thickness (an int that determines how erased the notepad lines are)
# Returns a modified image, where the notepad lines are attempted to be erased
def remove_lines(input_image, thickness=2):
  edges = cv2.Canny(input_image, 200, 200)
  lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

  # Creates a mask based on the lines found by HoughLines
  line_mask = np.zeros_like(input_image)
  for line in lines:
    rho, theta = line[0]
    if theta < 1.0 or theta > 2.0:
      continue
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + line_length * (-b))
    y1 = int(y0 + line_length * (a))
    x2 = int(x0 - line_length * (-b))
    y2 = int(y0 - line_length * (a))
    cv2.line(line_mask, (x1, y1), (x2, y2), (255, 255, 255), 2)

  # Dilates the lines vertically based on thickness parameter
  dilation_kernel = np.ones((thickness, 1),np.uint8)
  line_mask = cv2.dilate(line_mask, dilation_kernel, iterations=1)

  # Subtracts the mask from the base image and applies MORPH_OPEN to denoise
  sub_result = cv2.bitwise_or(input_image, line_mask)
  final_result = cv2.bitwise_not(cv2.morphologyEx(cv2.bitwise_not(sub_result), cv2.MORPH_OPEN, np.ones((2, 2),np.uint8), iterations=1))
  return final_result

### ROI Extraction Function

In [10]:
# Takes in an image from which contours are found, an image for OCR, and target number of ROIs
# Returns an array of RIOs (of shape [y, x]), and a boolean for error handling
def extract_rois(contour_image, ocr_image, num_of_items):
  # -------------------------------- FOR GETTING CONTOURS -------------------------------- #
  all_contours = cv2.findContours(contour_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  all_contours = all_contours[0] if len(all_contours) == 2 else all_contours[1]
  all_contours = sorted(all_contours, key=lambda x: (cv2.boundingRect(x)[1], cv2.boundingRect(x)[0]))

  # Filter contours by size
  filtered_contours = []
  x_coordinates = []
  for contour in all_contours:
    x, y, w, h = cv2.boundingRect(contour)
    if h > min_contour_height and w > min_contour_width and h < max_contour_height and w < max_contour_width:
      filtered_contours.append(contour)
      x_coordinates.append(int(x + (w / 2)))

  # -------------------------------- FOR GROUPING CONTOURS -------------------------------- #
  # Group contours based on x position
  x_coordinates = np.array(x_coordinates).reshape(-1, 1)

  # Perform K-means clustering with k = number_of_columns
  optimal_k = number_of_columns
  kmeans = KMeans(n_clusters=optimal_k, init='k-means++', max_iter=300, n_init=10, random_state=0)
  kmeans.fit(x_coordinates)

  # Group the contours based on the cluster assignments
  grouped_contours = [[] for _ in range(optimal_k)]
  for i, label in enumerate(kmeans.labels_):
    grouped_contours[label].append(filtered_contours[i])
  grouped_contours = sorted(grouped_contours, key=lambda group: cv2.boundingRect(group[0])[0])

  # ---------------------------------- FOR GETTING ROIs ---------------------------------- #
  # Get a list of ROIs to filter non-letters based on relative position
  roi_x_dict = {}
  roi_y_dict = {}
  roi_boxes = {}
  counter = 0
  for group in grouped_contours:
    for countour in group:
      counter += 1
      x, y, w, h = cv2.boundingRect(countour)
      roi_x_dict[counter] = x
      roi_y_dict[counter] = y
      roi_boxes[counter] = (x, y, w, h)

  # Filter ROIs by Y position, compared to nearby ROIs (prev, prev-prev, next)
  duplicate_dict = roi_y_dict.copy()
  for key in duplicate_dict:
    prev_prev_key = key - 2
    prev_key = key - 1
    next_key = key + 1
    if not key in roi_y_dict:
      continue

    if prev_key in roi_y_dict:
      if abs(duplicate_dict[prev_key] - duplicate_dict[key]) <= y_epsilon:
        prev_key_x = roi_x_dict[prev_key]
        curr_key_x = roi_x_dict[key]
        if prev_key_x > curr_key_x:
          if key in roi_y_dict:
            roi_y_dict.pop(key)
            continue

    if prev_prev_key in roi_y_dict:
      if abs(duplicate_dict[prev_prev_key] - duplicate_dict[key]) <= y_epsilon:
        prev_prev_key_x = roi_x_dict[prev_prev_key]
        curr_key_x = roi_x_dict[key]
        if prev_prev_key_x > curr_key_x:
          if key in roi_y_dict:
            roi_y_dict.pop(key)
            continue

    if next_key in roi_y_dict:
      if abs(duplicate_dict[next_key] - duplicate_dict[key]) <= y_epsilon:
        next_key_x = roi_x_dict[next_key]
        curr_key_x = roi_x_dict[key]
        if next_key_x > curr_key_x:
          if key in roi_y_dict:
            roi_y_dict.pop(key)
            continue

  # Handle exception: If the number of ROIs found is not the same as the target, consider the image invalid
  if len(roi_y_dict) != num_of_items:
    return [0], True

  # Generate the array of ROIs
  rois = []
  counter = 0
  for roi in roi_y_dict:
    if roi in roi_boxes:
      x, y, w, h = roi_boxes[roi]
      counter += 1
      roi_img = ocr_image[y:y+h, x:x+w]
      rois.append(roi_img)

  return rois, False

### Image Classification Function

In [11]:
# Takes in an ROI expressed as a numpy array of shape [y, x]
# Returns the classifiers classification of the ROI
def classify_roi(roi):
  # Preprocess ROI
  roi = cv2.resize(roi, (28, 28)) # Resize to fit model requirement
  roi = cv2.bitwise_not(roi) # Invert to fit model requirement
  roi = roi / 255.0 # Normalize

  roi = torch.from_numpy(roi) # Convert to tensor
  roi = roi.view(1, 1, 28, 28) # Reshape to fit model requirement
  roi = roi.to(torch.float32) # Change data type to fit model requirement

  # Classify the ROI
  model.eval()
  output = model(roi)
  prediction = output.argmax(dim=1, keepdim=True).item() # Returns a number from 0 to 3
  predicted_letter = PREDICTION_TO_STRING[prediction] # Converts the number to the corresponding letter
  return predicted_letter

## THE INTERFACE
Use Gradio to create a user-friendly interface.

In [12]:
# Download the sample images for the demo interface
SAMPLE_IMAGES_FOLDER_URL = "https://raw.githubusercontent.com/CorvusBrachyrhynchos/Handwritten-Multiple-Choice-Test-Corrector/main/sample_images/"
IMAGE_SAVE_PATH = "/content/"

NUM_OF_SAMPLES = 2
for i in range(1, NUM_OF_SAMPLES + 1):
  suffix = f"sample_{i}.jpg"
  os.system(f"wget {SAMPLE_IMAGES_FOLDER_URL}{suffix} -O {IMAGE_SAVE_PATH}{suffix}")

In [ ]:
# Define the interface function
def interface_function(correction_key_string, *images):

  # Set correction key
  correction_key = []
  for item in correction_key_string:
    if item in ["A", "B", "C", "D", "X"]:
      correction_key.append(item.upper())

  # Run the main function
  image_scores, score_analysis, item_analysis = correct_image_set(images[0], correction_key)

  # Generate output strings
  scores_string = ""
  score_analysis_string = ""
  item_analysis_string = ""

  num_of_items = len(correction_key)
  num_of_students = len(images[0])

  for index, score in enumerate(image_scores, start=1):
    scores_string += f"Student {index}: {score}/{num_of_items}\n"

  for index, num_of_scores in enumerate(score_analysis):
    score_analysis_string += f"{index}/{num_of_items}: {num_of_scores}\n"

  for index, item_score in enumerate(item_analysis, start=1):
    item_analysis_string += f"Item {index}: {item_score}/{num_of_students}\n"

  return scores_string, score_analysis_string, item_analysis_string


# Define the Gradio interface

# Set the description string
desc ='''
This is the demo interface for the research project titled "AUTOMATING MULTIPLE-CHOICE HANDWRITTEN TEST CORRECTION USING COMPUTER VISION AND DEEP LEARNING" by Edradan, G., Serrano, D., and Tunguia, T.

Instructions:

First Input: Enter the correction key. It must be a continuous string of text, with each letter representing the correct answer for each item in consecutive order. The only letters accepted are A, B, C, and D, but an X can be written to represent an item that would accept any answer (bonus item).

Second Input: Upload the images of the papers to be evaluated and analyzed. The order of evaluation is based on the order in which the images are uploaded.

For better results, the following are recommended:

Regarding the documents to be evaluated:
  - Have substantial left and right margins
  - Provide a blank line between the header and the answers
  - Write the answers in capitals and in two columns
  - Avoid having the answers overlap with the notepad lines
  - Have significant space between the numbers and the letters
  - Write the item numbers smaller than the letters

Regarding the photo:
  - Have an aspect ratio of 3:4
  - Have a resolution of at least 1125 px by 1500 px
  - Have adequate lighting; use flash if necessary

'''

interface = gr.Interface(
    fn=interface_function,
    title="HANDWRITTEN MULTIPLE-CHOICE TEST EVALUATOR",
    description=desc,
    allow_flagging="never",
    inputs=[gr.Textbox(label="Correction Key", placeholder="ABCDABCDABCD"),
            gr.File(file_count="multiple", file_types=[".jpg"], label="Upload Image(s)")],
    outputs=[gr.Textbox(label="Scores"),
             gr.Textbox(label="Score Analysis"),
             gr.Textbox(label="Item Analysis")],
    examples=[
        ["ABCDXABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDA",(['sample_1.jpg'])],
        ["ABCDXABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDA",(['sample_2.jpg'])],
        ["ABCDXABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDA",(['sample_1.jpg', 'sample_2.jpg'])],
    ]
)


# Launch the interface
interface.launch(debug=False)